In [336]:
import json
import gurobipy as gp
from gurobipy import GRB

with open('parametros_probando.json', encoding='utf8') as f:
    data = json.load(f)

cantidad_de_horas_por_ramo_por_curso = data['cantidad_de_horas_por_ramo_por_curso']
ramos_y_cursos_dictados_por_profesor = data['ramos_y_cursos_dictados_por_profesor']
disponibilidad_horaria = data['disponibilidad_horaria']
clases_ya_asignadas = data['clases_ya_asignadas']
preferencias_profesores = data['preferencias_profesores']

# Se necesita saber a qué cursos PUEDE (nada de querer todavía) hacer clases cada profesor, dada las asignaciones previas que ya tiene. 

In [337]:
# ! Formar tuplas profesores-ramos-cursos

def tuplas_ramos_cursos(cantidad_de_horas_por_ramo_por_curso):
    lista_tuplas = []
    for c in cantidad_de_horas_por_ramo_por_curso:
        for r in cantidad_de_horas_por_ramo_por_curso[c]:
            lista_tuplas.append((r,c))
    return lista_tuplas

def tuplas_profesores_ramos_cursos(cantidad_de_horas_por_ramo_por_curso):
    lista_trios = []
    for p in ramos_y_cursos_dictados_por_profesor:
        for r,c in ramos_y_cursos_dictados_por_profesor[p]:
            lista_trios.append((p,r,c))
    return lista_trios

In [338]:
model = gp.Model("Preasignación de cursos a profesores")

cursos = list(cantidad_de_horas_por_ramo_por_curso.keys())
profesores = list(ramos_y_cursos_dictados_por_profesor.keys())
# ramos = ['Matemáticas', 'Lenguaje', 'Historia', 'Alemán']

duplas_rc = tuplas_ramos_cursos(cantidad_de_horas_por_ramo_por_curso)
trios_prc = tuplas_profesores_ramos_cursos(cantidad_de_horas_por_ramo_por_curso)

# ! Otros conjuntos importantes

y = model.addVars(trios_prc, vtype=GRB.BINARY, name="y")
w = model.addVars(duplas_rc, vtype=GRB.BINARY, name = "w")

In [339]:
model.addConstrs((sum(y[p,r,c] for p,s,d in trios_prc if r == s if c == d) + w[r,c] == 1 for r,c in duplas_rc),name="R1");

In [340]:
model.addConstrs((sum(y[p,r,c] * cantidad_de_horas_por_ramo_por_curso[c][r] for p,r,c in trios_prc if p == q) <= disponibilidad_horaria[q] for q in ramos_y_cursos_dictados_por_profesor),name="R2");

In [341]:
model.addConstrs((y[p,r,c] == 1 for r,p,c in clases_ya_asignadas),name="R3");

In [343]:
obj = 50 * sum(w[r,c] * cantidad_de_horas_por_ramo_por_curso[c][r] for c in cantidad_de_horas_por_ramo_por_curso for r in cantidad_de_horas_por_ramo_por_curso[c]) # ? Minimizar la cantidad de cursos asignados que hay que parchar
model.setObjective(obj, GRB.MINIMIZE)


model.update()
model.write('model.lp')
# model.computeIIS()
# removed =[]
# for c in model.getConstrs():
#     if c.IISConstr:
#         print('%s' % c.constrName)
#         # Remove a single constraint from the model
#         removed.append(str(c.constrName))
#         model.remove(c)

model.optimize()
model.write('out_preasignacion.sol')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 832 rows, 22495 columns and 44478 nonzeros
Model fingerprint: 0x530d71a6
Variable types: 0 continuous, 22495 integer (22495 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [5e+01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 79950.000000
Presolve removed 200 rows and 3769 columns
Presolve time: 0.06s
Presolved: 632 rows, 18726 columns, 36940 nonzeros
Variable types: 0 continuous, 18726 integer (18726 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.08 seconds (0.06 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 0 79950 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [344]:
with open ('out_preasignacion.sol') as f:
    profesores_faltantes = []
    ocupaciones_profesores = []
    for line in f.readlines()[2:]:
        if line[-2] == '1' and line[0] == 'w':
            line = line.strip('\n')
            line = line[2:-3]
            line = line.split(',')
            profesores_faltantes.append(line)
        if line[-2] == '1' and line[0] == 'y':
            line = line.strip('\n')
            line = line[2:-3]
            line = line.split(',')
            ocupaciones_profesores.append(line)

suma = 0
for elem in profesores_faltantes:
    suma += cantidad_de_horas_por_ramo_por_curso[elem[1]][elem[0]]
    print(f'Al curso {elem[1]} no fue posible asignarle profesor de {elem[0]} ({cantidad_de_horas_por_ramo_por_curso[elem[1]][elem[0]]} horas)')
print(f'El total de horas que no se pudieron asignar fue de {suma} horas')
horas_placeholder = 1.5 * suma # ! Esta es una estimación que considera la cantidad de horas placeholder 
                               # ! que faltarían y a la vez deja espacio para las prioridades


suma = 0
for i in range(len(ocupaciones_profesores)):
    if i+1 != len(ocupaciones_profesores):
        if ocupaciones_profesores[i][0] == ocupaciones_profesores[i+1][0]:
            suma += cantidad_de_horas_por_ramo_por_curso[ocupaciones_profesores[i][2]][ocupaciones_profesores[i][1]]
        if ocupaciones_profesores[i][0] != ocupaciones_profesores[i+1][0]:
            suma += cantidad_de_horas_por_ramo_por_curso[ocupaciones_profesores[i][2]][ocupaciones_profesores[i][1]]
            print(f'Profesor {ocupaciones_profesores[i][0]} tiene {suma} horas asignadas')
            suma = 0
    if i+1 == len(ocupaciones_profesores):
        suma += cantidad_de_horas_por_ramo_por_curso[ocupaciones_profesores[i][2]][ocupaciones_profesores[i][1]]
        print(f'Profesor {ocupaciones_profesores[i][0]} tiene {suma} horas asignadas')


El total de horas que no se pudieron asignar fue de 0 horas
Profesor Rlybywxl tiene 20 horas asignadas
Profesor Lhguzsdp tiene 20 horas asignadas
Profesor Atzgrmcj tiene 20 horas asignadas
Profesor Dwsfwtcu tiene 20 horas asignadas
Profesor Uskngdik tiene 20 horas asignadas
Profesor Fpqpcfbq tiene 20 horas asignadas
Profesor Kvqamngw tiene 20 horas asignadas
Profesor Xfhyxhmf tiene 20 horas asignadas
Profesor Ipgxqoqq tiene 20 horas asignadas
Profesor Ulvuyjza tiene 20 horas asignadas
Profesor Xfyryhan tiene 20 horas asignadas
Profesor Mmbudkeu tiene 20 horas asignadas
Profesor Cdinnylg tiene 20 horas asignadas
Profesor Nolmbzqj tiene 20 horas asignadas
Profesor Vvfythjg tiene 20 horas asignadas
Profesor Bzbkyeqh tiene 20 horas asignadas
Profesor Atkvuhru tiene 20 horas asignadas
Profesor Yiqzsxsm tiene 20 horas asignadas
Profesor Oueurwlr tiene 20 horas asignadas
Profesor Xwmgnbka tiene 20 horas asignadas
Profesor Sgnzggdl tiene 20 horas asignadas
Profesor Ylbruxtg tiene 20 horas asig

In [345]:
model2 = gp.Model("Asignación de cursos a profesores")

prc_para_model2 = trios_prc.copy()
tuplas_placeholder = []
for r,c in duplas_rc:
    tuplas_placeholder.append(('Placeholder', r, c))
tuplas_placeholder

prc_placeholder = prc_para_model2 + tuplas_placeholder

z = model2.addVars(prc_placeholder, vtype=GRB.BINARY, name="z")

In [346]:
model2.addConstrs((sum(z[p,r,c] for p,s,d in prc_placeholder if r == s if c == d) == 1 for r,c in duplas_rc),name="R1");

In [347]:
model2.addConstrs((sum(z[p,r,c] * cantidad_de_horas_por_ramo_por_curso[c][r] for p,r,c in trios_prc if p == q) \
    <= disponibilidad_horaria[q] for q in ramos_y_cursos_dictados_por_profesor),name="R2");

In [348]:
model2.addConstr((sum(z['Placeholder',r,c] * cantidad_de_horas_por_ramo_por_curso[c][r] for r,c in duplas_rc) \
    <= horas_placeholder),name="R3");

In [349]:
model2.addConstrs((z[p,r,c] == 1 for r,p,c in clases_ya_asignadas),name="R5");

In [351]:
obj2 = 3 * sum(z[p,r,c] for p in profesores for r,c in preferencias_profesores[p]) - 2 * sum(z['Placeholder',r,c] for r,c in duplas_rc)
model2.setObjective(obj2, GRB.MAXIMIZE)

model2.write('model.lp')
model2.update()
model2.optimize()
model2.write('out.sol')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 833 rows, 22495 columns and 45090 nonzeros
Model fingerprint: 0x0ba4624a
Variable types: 0 continuous, 22495 integer (22495 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]

Loaded MIP start from previous solve with objective 762

Presolve removed 201 rows and 4281 columns
Presolve time: 0.06s
Presolved: 632 rows, 18214 columns, 36428 nonzeros
Variable types: 0 continuous, 18214 integer (18214 binary)

Root relaxation: objective 7.734286e+02, 681 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  773.42857    0   14  762.00000  773.42857  1.50%     -    0s
     0 

In [352]:
with open ('out.sol') as f:
    ocupaciones_profesores = []
    for line in f.readlines()[2:]:
        if line[-2] == '1' and line[0] == 'z':
            line = line.strip('\n')
            line = line[2:-3]
            line = line.split(',')
            ocupaciones_profesores.append(line)

for elem in ocupaciones_profesores:
    # if elem[0] != 'Placeholder':
    #     if (elem[1], elem[2]) in preferencias_profesores[elem[0]]:
    #         print(elem)
    if elem[0] == 'Placeholder':
        print(elem)

In [353]:
def generar_profesores_por_curso(ocupaciones_profesores):
    ocupaciones_profesores = sorted(ocupaciones_profesores, key= lambda x: x[1])
    ocupaciones_profesores = sorted(ocupaciones_profesores, key= lambda x: x[2])

    ultimo_ramo = None
    ultimo_curso = None
    indices = []
    for i in range(len(ocupaciones_profesores)):
        if ocupaciones_profesores[i][1] == ultimo_ramo:
            if ocupaciones_profesores[i][2] == ultimo_curso:
                indices.append(i)
        ultimo_ramo = ocupaciones_profesores[i][1]
        ultimo_curso = ocupaciones_profesores[i][2]
    indices.reverse()

    curso_actual = None
    profesores_por_curso = {}
    for i in range(len(ocupaciones_profesores)):
        if ocupaciones_profesores[i][2] != curso_actual:
            profesores_por_curso[ocupaciones_profesores[i][2]] = {}
            if type(ocupaciones_profesores[i][0]) == str:
                profesores_por_curso[ocupaciones_profesores[i][2]][1] = [[ocupaciones_profesores[i][0], ocupaciones_profesores[i][1]]]
            else:
                profesores_por_curso[ocupaciones_profesores[i][2]][len(ocupaciones_profesores[i][0])] = [[ocupaciones_profesores[i][0], ocupaciones_profesores[i][1]]]
        else:
            if type(ocupaciones_profesores[i][0]) == str:
                if 1 not in profesores_por_curso[ocupaciones_profesores[i][2]]:
                    profesores_por_curso[ocupaciones_profesores[i][2]][1] = [[ocupaciones_profesores[i][0], ocupaciones_profesores[i][1]]]
                else:
                    profesores_por_curso[ocupaciones_profesores[i][2]][1].append([ocupaciones_profesores[i][0], ocupaciones_profesores[i][1]])
            else:
                if len(ocupaciones_profesores[i][0]) not in profesores_por_curso[ocupaciones_profesores[i][2]]:
                    profesores_por_curso[ocupaciones_profesores[i][2]][len(ocupaciones_profesores[i][0])] = [[ocupaciones_profesores[i][0], ocupaciones_profesores[i][1]]]
                else:
                    profesores_por_curso[ocupaciones_profesores[i][2]][len(ocupaciones_profesores[i][0])].append([ocupaciones_profesores[i][0], ocupaciones_profesores[i][1]])
        curso_actual = ocupaciones_profesores[i][2]
    return profesores_por_curso

def generar_carga_profesores(ocupaciones_profesores):
    ocupaciones_profesores = sorted(ocupaciones_profesores, key= lambda x: x[0])
    carga_profesores = {}
    profesor_actual = None
    for i in range(len(ocupaciones_profesores)):
        if ocupaciones_profesores[i][0] != profesor_actual:
            carga_profesores[ocupaciones_profesores[i][0]] = {}
            carga_profesores[ocupaciones_profesores[i][0]]["1"] = [[ocupaciones_profesores[i][2], ocupaciones_profesores[i][1]]]
        else:
            carga_profesores[ocupaciones_profesores[i][0]]["1"].append([ocupaciones_profesores[i][2], ocupaciones_profesores[i][1]])
        profesor_actual = ocupaciones_profesores[i][0]
    return carga_profesores


output = {}
output['profesores_por_curso'] = generar_profesores_por_curso(ocupaciones_profesores)
output['ramos_por_curso'] = cantidad_de_horas_por_ramo_por_curso
output['carga_profesores'] = generar_carga_profesores(ocupaciones_profesores)
with open('probando.json', 'w', encoding='utf8') as f:
    json.dump(output, f, indent=4)